# Using Dask-ML's CountVectorizer

Dask-ML includes a [CountVectorizer](https://ml.dask.org/modules/generated/dask_ml.feature_extraction.text.CountVectorizer.html#dask_ml.feature_extraction.text.CountVectorizer) that's appropriate for parallel / distributed processing of large datasets.

## Loading Data

As we'll see later, Dask-ML's `CountVectorizer` benefits from using the `dask.distributed` scheduler, even on a single machine.

In [ ]:
from dask.distributed import Client

client = Client(n_workers=4, threads_per_worker=1)
client

In this example, we'll work with the 20 newsgroups dataset from scikit-learn.

In [ ]:
import sklearn.datasets

news = sklearn.datasets.fetch_20newsgroups()
news['data'][:2]

This returns a list of documents (strings). Dask-ML's `CountVectorizer` expects a `dask.bag.Bag` of documents. We'll use `dask.delayed` to load the 20 newsgroups in parallel, taking care to load the data on the workers and not place large values (like `news['data']`) in the the task graph. See https://docs.dask.org/en/latest/best-practices.html#load-data-with-dask and https://docs.dask.org/en/latest/delayed-best-practices.html#don-t-call-dask-delayed-on-other-dask-collections for more on these concepts.

This example is a bit contrived to get a Bag with multiple partitions. Typically the full dataset would be partitioned into multiple files on disk, and you'd load one partition per file. In this case, we split the single file into multiple partitions by loading the data and then slicing.

In [ ]:
import dask
import numpy as np
import dask.bag as db
import toolz

@dask.delayed
def load_news(slice_):
    """Load a slice of the 20 newsgroups dataset."""
    return sklearn.datasets.fetch_20newsgroups()['data'][slice_]

npartitions = 10
partition_size = len(news['data']) // npartitions

lengths = np.cumsum([partition_size] * npartitions)
lengths = [0] + list(lengths) + [None]

slices = [slice(a, b) for a, b in
          toolz.sliding_window(2, lengths)]
# Notice the persist here! More details later.
documents = db.from_delayed([load_news(x) for x in slices]).persist()
documents

In [ ]:
import dask_ml.feature_extraction.text

In [ ]:
vectorizer = dask_ml.feature_extraction.text.CountVectorizer()
%time result = vectorizer.fit_transform(documents)

The call to `fit_transform` did some work to discover the *vocabulary*, a mapping from terms in the documents to positions in the transformed result array.

In [ ]:
list(vectorizer.vocabulary_.items())[:5]

Speaking of the result, it's a Dask `Array` backed by `scipy.sparse.csr_matrix` objects. We can bring it back to the client with `.compute()`

In [ ]:
local_result = result.compute()
local_result[:5].toarray()

Notice that we persisted `documents` earlier. If possible, persisting the input documents is preferable to avoid making two passes over the data. One to discover the vocabulary and a second to transform. If the dataset is larger than (distributed) memory, then two passes will be necessary.

##  A note on vocabularies

You can also provide a vocabulary ahead of time, which avoids the need for making two passes over the data. This makes operations like `vectorizer.transform` instantaneous, since no vocabulary needs to be discovered. However, vocabularies can become quite large. Consider persisting your data ahead of time to avoid bloating the size of the `CountVectorizer` object. Dask-ML's `CountVectorizer` works just fine when the `vocabulary` is a pointer to a piece of data on the cluster.

In [ ]:
vocabulary = vectorizer.vocabulary_
remote_vocabulary, = client.scatter([vocabulary], broadcast=True)

vectorizer2 = dask_ml.feature_extraction.text.CountVectorizer(
    vocabulary=remote_vocabulary
)

In [ ]:
%time result = vectorizer2.transform(documents)

In [ ]:
%time result.compute()